# Finding brain regions and getting their center of mass using the Allen SDK

1. Create a new python virtual environment or use an existing one.
1. pip install allensdk==2.15.1 # this works with the current ubuntu/pipeline we have, you can try just doing pip install allensdk
1. The Allen data has to be download which happens in the commands below. You might get an error on downloading, if you do, just copy the link that is in the error message and use curl or wget or just put that link in the browser. It will then download. You will need to move that into the current working directory of this notebook under the 'mouse_connectivity' directory
1. You'll notice the atlas below has the rostrum at the top. These are the coordinates of the Allen atlas. We need the rostrum at the left, with the superior at the top in the sagittal view. So the center of mass coordinates you get below need to be flipped.


In [78]:
import numpy as np
import pandas as pd
from scipy.ndimage import center_of_mass
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
from matplotlib import pyplot as plt

In [79]:
%matplotlib notebook

### Accessing the CCF from the MouseConnectivityCache

Instantiate the MouseConnectivityCache (mcc). The default resolution is 25 microns, but 10,50, and 100 microns are also valid options. You can also specify a location of a manifest file, which would keep track of all downloaded assets from template and annotation volumes to experimental data. If a manifest file directory is not provided, one will be created in the current working directory. (This is where you might get downloading errors, just download it manually and place it under the dir: mouse_connectivity'

In [80]:
mcc = MouseConnectivityCache(resolution=25)

#### Annotated Volume and Reference Space
Now you can get the Reference Space (rsp) at that resolution. This automatically downloads the annotated volume at the specified resolution (25 microns).

In [81]:
rsp = mcc.get_reference_space()

The annotated volume is a 3D numpy ndarray, with axes correspond to AP,DV, and ML respectively:

In [82]:
print('Type of annotated volume', type(rsp))
print('Shape of annotated volume:',rsp.annotation.shape)
print('Annotated volume data type:',rsp.annotation.dtype)

Type of annotated volume <class 'allensdk.core.reference_space.ReferenceSpace'>
Shape of annotated volume: (528, 320, 456)
Annotated volume data type: uint32


The rsp provides access to the structure tree from the ontology. Given that there are multiple atlases and associated ontologies, we are only interested in the structures belonging to this annotation. Thus, we remove unassigned structures from the structure tree. This returns a structure graph which is more easily viewed as a pandas DataFrame:

In [83]:
sg = rsp.remove_unassigned()

In [84]:
sg = pd.DataFrame(sg)

In [85]:
sg.head()

,acronym,graph_id,graph_order,id,name,structure_id_path,structure_set_ids,rgb_triplet
0,root,1,0,997,root,[997],[691663206],"[255, 255, 255]"
1,grey,1,1,8,Basic cell groups and regions,"[997, 8]","[112905828, 691663206, 12, 184527634, 11290581...","[191, 218, 227]"
2,CH,1,2,567,Cerebrum,"[997, 8, 567]","[112905828, 691663206, 12, 184527634, 11290581...","[176, 240, 255]"
3,CTX,1,3,688,Cerebral cortex,"[997, 8, 567, 688]","[112905828, 691663206, 12, 184527634, 11290581...","[176, 255, 184]"
4,CTXpl,1,4,695,Cortical plate,"[997, 8, 567, 688, 695]","[112905828, 691663206, 12, 184527634, 11290581...","[112, 255, 112]"


In [92]:
# I picked the cerebellum as it is easy to see
# get data with structure name
structure = 'SC'
row = sg.loc[sg['name'] == structure]
row

,acronym,graph_id,graph_order,id,name,structure_id_path,structure_set_ids,rgb_triplet


In [86]:
# get data with acronym
acronym = 'SCs'
row = sg.loc[sg['acronym'] == acronym]
row

,acronym,graph_id,graph_order,id,name,structure_id_path,structure_set_ids,rgb_triplet
536,SCs,1,808,302,"Superior colliculus, sensory related","[997, 8, 343, 313, 339, 302]","[112905828, 691663206, 687527945, 10, 12, 6881...","[255, 122, 255]"


In [44]:
sg.query('name.str.contains("facial")', engine='python')

,acronym,graph_id,graph_order,id,name,structure_id_path,structure_set_ids,rgb_triplet
656,ACVII,1,968,576,Accessory facial motor nucleus,"[997, 8, 343, 1065, 354, 370, 576]","[112905828, 691663206, 687527945, 12, 68815236...","[255, 179, 217]"
745,VIIn,1,1130,798,facial nerve,"[997, 1009, 967, 798]","[184527634, 691663206]","[204, 204, 204]"
746,gVIIn,1,1132,1116,genu of the facial nerve,"[997, 1009, 967, 798, 1116]","[184527634, 691663206]","[204, 204, 204]"


In [87]:
structure_id = row.iat[0, 3]

In [88]:
# Get a list of acronymns
id_acronym_map = rsp.structure_tree.get_id_acronym_map() # Dictionary returns IDs given acronyms
acronym_id_map = {v:k for k,v in id_acronym_map.items()} # Flip key:value pairs to get dictionary for acronyms given IDs
print(list(map(acronym_id_map.get,rsp.structure_tree.descendant_ids([structure_id])[0])))

['SCs', 'SCop', 'SCsg', 'SCzo']


The structures' acronyms, names and IDs can all be linked through the structure graph.

While the rsp can be used to generate sections in the coronal (0), horizontal (1), or sagittal (2) planes:

In [89]:
orientation = {'coronal':0,'horizontal':1,'sagittal':2} # makes orientation arguments more readible
pos_microns = lambda x:x*25 # get_slice_image function takes positional arguments in microns instead of 25 micron increments

img = rsp.get_slice_image(orientation['sagittal'],pos_microns(200))
plt.imshow(img)

The rsp can also generate masks of given structure IDs. The IDs are provided as a list, and by default include all voxels annotated as children of the given structures (direct_only=False). If only the parent level voxels are desired, set direct_only to True.

Here we make a mask of the Cerebral Cortex, which from the structure graph is shown to have the ID 688.

In [90]:
structure_mask = rsp.make_structure_mask([structure_id], direct_only=False)

In [93]:
print('Shape of entire brain', rsp.annotation.shape)
print('Shape of mask:',structure_mask.shape)
print('Mask data type:',structure_mask.dtype)
values, counts = np.unique(structure_mask, return_counts=True)
com = center_of_mass(structure_mask)
print(f'Center of mass for {structure}: {com}')

Shape of entire brain (528, 320, 456)
Shape of mask: (528, 320, 456)
Mask data type: uint8
Center of mass for SC: (368.08973894913635, 62.21607279887645, 227.68008600344592)


In [77]:
com_section = int(round(com[2])) 
plt.scatter([com[1]], [com[0]], c='r', s=40)
plt.imshow(structure_mask[:,:,com_section])